In [17]:
#Import libraries, load watermark and set directories
from atg import *
from tqdm import tqdm

In [3]:
PATH=""
GIRD_FILE="GHS_POP_E2030_GLOBE_R2023A_4326_3ss_V1_0.tif"
GRID_CRS="4326"
BUFFER_SIZE=3000

### Save Raster for DB

In [ ]:
os.chdir(PATH)
db_params=pickle.load(open(f"{PATH}/dicts/db_params.pickle", 'rb'))
cities_list=query4table('cities_boundary','public', db_params)['city']

#Geometry for clip
for ind, city in tqdm(enumerate(cities_list)):
    geometry_to_clip=query4cityboundary(city, db_params, BUFFER_SIZE).geometry

    #Get Clipped raster
    xds=getClippedRaster("population", f"{PATH}/grid/{GRID_FILE}", geometry_to_clip, True, True, "./", city)

    #Generate to SQL table
    if ind==0:
        mode="d"
    else:
        mode="a"

    rast2sql(".", city ,"tmp", "ghs_pop", mode, (xds.shape[2],xds.shape[1]), 'tiff', GRID_CRS)
    sqlRasterTable2db('ghs_pop','tmp',db_params)

    #Remove local file
    os.remove(f"{city}.tiff")